In [ ]:

!pip install prophet pandas matplotlib


In [ ]:

import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd


In [ ]:

df=pd.read_csv('/content/drive/MyDrive/Sample - Superstore (1).csv', encoding='latin1')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [ ]:
# Ensure datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Year'] = df['Order Date'].dt.year
df['Month'] = df['Order Date'].dt.month
df['Year-Month'] = df['Order Date'].dt.to_period('M').astype(str)


In [ ]:

overall_monthly = df.groupby(['Year-Month','Year','Month']).agg({
    'Sales':'sum',
    'Profit':'sum',
    'Quantity':'sum'
}).reset_index().rename(columns={
    'Sales':'Actual Sales',
    'Profit':'Actual Profit',
    'Quantity':'Actual Quantity'
})



In [ ]:

# Prophet forecast
forecast_input = overall_monthly[['Year-Month','Actual Sales']].rename(columns={'Year-Month':'ds','Actual Sales':'y'})
forecast_input['ds'] = pd.to_datetime(forecast_input['ds'])

m = Prophet(interval_width=0.95, yearly_seasonality=True)
m.fit(forecast_input)

future = m.make_future_dataframe(periods=12, freq='M')
forecast = m.predict(future)

forecast = forecast[['ds','yhat','yhat_lower','yhat_upper']].rename(columns={
    'ds':'Date',
    'yhat':'Predicted Sales',
    'yhat_lower':'Predicted Sales (Lower Bound)',
    'yhat_upper':'Predicted Sales (Upper Bound)'
})
forecast['Year'] = forecast['Date'].dt.year
forecast['Month'] = forecast['Date'].dt.month
forecast['Year-Month'] = forecast['Date'].dt.to_period('M').astype(str)



AttributeError: 'Prophet' object has no attribute 'stan_backend'

In [ ]:
df.columns = df.columns.str.strip().str.lower()

monthly_category = df.groupby(
    ['year-month','year','month','category','region']
).agg({
    'y': 'sum',         # this is Sales
    'profit':'sum',
    'quantity':'sum'
}).reset_index().rename(columns={
    'y':'Actual Sales',
    'profit':'Actual Profit',
    'quantity':'Actual Quantity'
})

monthly_category['data_type'] = 'Category_Monthly'

print(monthly_category.head())



KeyError: 'year-month'

In [ ]:
# ensure lowercase columns
df.columns = df.columns.str.strip().str.lower()

top_products = df.groupby(['product name','category','region']).agg({
    'y':'sum',          # Sales column (y)
    'profit':'sum',
    'quantity':'sum'
}).reset_index().rename(columns={
    'y':'Product Sales',
    'profit':'Product Profit',
    'quantity':'Product Quantity'
}).sort_values('Product Sales', ascending=False).head(10)

top_products['Rank'] = range(1, len(top_products) + 1)
top_products['Data_Type'] = 'Top10_Product'

top_products


In [ ]:
df.columns = df.columns.str.strip().str.lower()

yearly_totals = df.groupby('year').agg({
    'y':'sum',          # sales column = y
    'profit':'sum',
    'quantity':'sum'
}).reset_index().rename(columns={
    'y':'Yearly Actual Sales',
    'profit':'Yearly Actual Profit',
    'quantity':'Yearly Actual Quantity'
})


In [ ]:

overall_block = overall_monthly.merge(forecast, on='Year-Month', how='left')
overall_block['Category'] = 'All'
overall_block['Region'] = 'All'
overall_block['Data_Type'] = 'Overall_Monthly'

In [ ]:
# Keep consistent column order
final_cols = [
    'Data_Type','Date','Year','Month','Year-Month','Category','Region','Product Name',
    'Actual Sales','Actual Profit','Actual Quantity',
    'Predicted Sales','Predicted Sales (Lower Bound)','Predicted Sales (Upper Bound)',
    'Product Sales','Product Profit','Product Quantity','Rank',
    'Yearly Actual Sales','Yearly Actual Profit','Yearly Actual Quantity',
    'Yearly Predicted Sales','Yearly Predicted Sales (Lower Bound)','Yearly Predicted Sales (Upper Bound)'
]

# Add missing columns with NA
def fix_cols(df, cols):
    for c in cols:
        if c not in df.columns:
            df[c] = pd.NA
    return df[cols]

combined = pd.concat([
    fix_cols(overall_block, final_cols),
    fix_cols(monthly_category, final_cols),
    fix_cols(top_products, final_cols),
    fix_cols(yearly_totals, final_cols)
], ignore_index=True)

# Convert Date to string for CSV
combined['Date'] = pd.to_datetime(combined['Date'], errors='coerce').dt.strftime('%Y-%m-%d')

In [ ]:

output_filename = "Sales_Forecast_Final.csv"
combined.to_csv(output_filename, index=False)
print("✅ Final CSV saved:", output_filename)

# Colab download
try:
    from google.colab import files
    files.download(output_filename)
except:
    print("File saved locally:", output_filename)

try 2

In [ ]:

!pip install prophet

In [ ]:

# Cell 2: Import libraries
import pandas as pd
from prophet import Prophet
from google.colab import files


In [ ]:

# Cell 3: Load dataset (you will upload and give path here)
# Example: df = pd.read_csv('/content/SampleSuperstore.csv')
df = pd.read_csv('/content/drive/MyDrive/Sample - Superstore (1).csv', encoding='latin1')

# Convert dates
df['Order Date'] = pd.to_datetime(df['Order Date'])

In [ ]:

# Cell 4: Create year and month features
df['year'] = df['Order Date'].dt.year
df['month'] = df['Order Date'].dt.month

In [ ]:

# Cell 5: Aggregate data at required level
agg_df = df.groupby(
    ['year','month','Region','Category','Sub-Category','Product Name'],
    as_index=False
).agg({
    'Sales':'sum',
    'Profit':'sum',
    'Quantity':'sum'
})

agg_df.rename(columns={
    'Region':'region',
    'Category':'category',
    'Sub-Category':'sub_category',
    'Product Name':'product_name',
    'Sales':'total_sales',
    'Profit':'total_profit',
    'Quantity':'quantity'
}, inplace=True)

In [ ]:

# Cell 6: Prepare data for forecasting (overall sales by month)
monthly_sales = df.groupby(pd.Grouper(key='Order Date', freq='M')).agg({'Sales':'sum'}).reset_index()
monthly_sales.rename(columns={'Order Date':'ds','Sales':'y'}, inplace=True)

In [ ]:

# Cell 7: Fit Prophet model
model = Prophet(interval_width=0.95)
model.fit(monthly_sales)

future = model.make_future_dataframe(periods=6, freq='M')  # 6 months forecast
forecast = model.predict(future)

In [ ]:
# Cell 8: Merge forecast with agg_df
# Extract year and month from forecast
forecast['year'] = forecast['ds'].dt.year
forecast['month'] = forecast['ds'].dt.month

forecast = forecast[['year','month','yhat','yhat_lower','yhat_upper']]
forecast.rename(columns={
    'yhat':'forecast_sales',
    'yhat_lower':'forecast_lower_ci',
    'yhat_upper':'forecast_upper_ci'
}, inplace=True)

# Add actual sales
actual = monthly_sales.copy()
actual['year'] = actual['ds'].dt.year
actual['month'] = actual['ds'].dt.month
actual = actual[['year','month','y']]
actual.rename(columns={'y':'actual_sales'}, inplace=True)

# Merge forecast + actual into agg_df
final_df = agg_df.merge(actual, on=['year','month'], how='left')
final_df = final_df.merge(forecast, on=['year','month'], how='left')


In [ ]:


# Cell 9: Save and download final CSV
final_df.to_csv('final_output.csv', index=False)
files.download('final_output.csv')